In [ ]:
import os
import numpy as np

# 프레임워크별 라이브러리 임포트
import torch  # PyTorch 사용 시
# import onnxruntime as ort  # ONNX 사용 시

# 텐서RT 관련 라이브러리 (실제 사용 시 추가 설정 필요)
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

ModuleNotFoundError: No module named 'onnxruntime'

In [ ]:
# config 변수: 모델과 프레임워크 선택
config = {
    "model": "simple",  # 옵션: "simple", "vit", "trajectory_prediction"
    "framework": "pytorch",  # 옵션: "pytorch", "onnx", "tensorrt"
}

In [ ]:
def load_model(config):
    """
    모델 파라미터 파일은 소스코드 폴더 내의 "models" 폴더에 저장되어 있다고 가정합니다.
    파일명은 예를 들어 "simple_model.pt", "vit_model.onnx", "trajectory_prediction_model.trt" 식으로 저장합니다.
    """
    model_type = config["model"]
    framework = config["framework"]
    model_dir = "models"
    
    if framework == "pytorch":
        model_path = os.path.join(model_dir, f"{model_type}_model.pt")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"PyTorch 모델 파일을 찾을 수 없습니다: {model_path}")
        model = torch.load(model_path, map_location=torch.device("cpu"))
        model.eval()
        return model

    elif framework == "onnx":
        model_path = os.path.join(model_dir, f"{model_type}_model.onnx")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"ONNX 모델 파일을 찾을 수 없습니다: {model_path}")
        session = ort.InferenceSession(model_path)
        return session

    elif framework == "tensorrt":
        model_path = os.path.join(model_dir, f"{model_type}_model.trt")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"TensorRT 모델 파일을 찾을 수 없습니다: {model_path}")
        TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
        runtime = trt.Runtime(TRT_LOGGER)
        with open(model_path, "rb") as f:
            engine_data = f.read()
        engine = runtime.deserialize_cuda_engine(engine_data)
        return engine

    else:
        raise ValueError("지원하지 않는 프레임워크입니다.")

In [ ]:
def get_input(config):
    """
    모델 종류에 맞는 입력 데이터를 생성합니다.
    각 모델은 예시로 랜덤 데이터를 사용하며,
    실제로는 모델이 기대하는 입력 형상과 데이터를 맞춰야 합니다.
    """
    model_type = config["model"]

    if model_type == "simple":
        # 예: 간단한 피드포워드 모델이 [1, 10] shape의 입력을 요구한다고 가정
        input_data = np.random.rand(1, 10).astype(np.float32)
    elif model_type == "vit":
        # 예: Vision Transformer가 [1, 3, 224, 224] shape의 이미지를 요구한다고 가정
        input_data = np.random.rand(1, 3, 224, 224).astype(np.float32)
    elif model_type == "trajectory_prediction":
        # 예: 궤적 예측 모델이 [1, 20, 2] shape의 시퀀스 데이터를 요구한다고 가정
        input_data = np.random.rand(1, 20, 2).astype(np.float32)
    else:
        raise ValueError("알 수 없는 모델 타입입니다.")

    return input_data

In [ ]:
def run_inference(config, model, input_data):
    """
    선택한 프레임워크에 따라 추론을 수행합니다.
    각 프레임워크에 맞는 전처리 및 실행 코드를 포함합니다.
    """
    framework = config["framework"]

    if framework == "pytorch":
        # PyTorch 추론
        with torch.no_grad():
            # numpy 데이터를 tensor로 변환
            input_tensor = torch.from_numpy(input_data)
            output = model(input_tensor)
            return output.detach().cpu().numpy()

    elif framework == "onnx":
        # ONNX 추론: 입력 이름을 가져와서 실행
        input_name = model.get_inputs()[0].name
        output = model.run(None, {input_name: input_data})
        return output

    elif framework == "tensorrt":
        # TensorRT 추론은 복잡합니다.
        # 아래는 매우 단순화한 예시로, 실제 사용 시 GPU 버퍼 할당 및 동기화 과정이 필요합니다.
        context = model.create_execution_context()

        # 입력 및 출력 버퍼 할당 (예시)
        input_shape = input_data.shape
        d_input = cuda.mem_alloc(input_data.nbytes)
        # 여기서는 출력의 shape와 크기를 미리 알아야 합니다.
        output_shape = (1, )  # 실제 모델에 맞게 수정 필요
        output_size = np.prod(output_shape) * np.float32().itemsize
        d_output = cuda.mem_alloc(output_size)

        # 입력 데이터 GPU로 복사
        cuda.memcpy_htod(d_input, input_data)

        # 실행 (바인딩 인덱스 0: 입력, 1: 출력라고 가정)
        bindings = [int(d_input), int(d_output)]
        context.execute_v2(bindings)

        # 결과를 CPU로 복사
        output = np.empty(output_shape, dtype=np.float32)
        cuda.memcpy_dtoh(output, d_output)
        return output

    else:
        raise ValueError("지원하지 않는 프레임워크입니다.")

In [ ]:
def main():
    # 모델 로드
    model = load_model(config)
    # 모델에 맞는 입력 데이터 생성
    input_data = get_input(config)
    # 추론 실행
    output = run_inference(config, model, input_data)
    print("Inference output:", output)

if __name__ == "__main__":
    main()

NameError: name 'os' is not defined